In [3]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime
import os
import shap
from lime import lime_tabular

# 생존 분석 라이브러리
from lifelines import CoxPHFitter, KaplanMeierFitter
from lifelines.statistics import logrank_test
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.ensemble import RandomSurvivalForest
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sksurv.metrics import concordance_index_censored

# 머신러닝 라이브러리
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, mean_squared_error, r2_score
import xgboost as xgb
import lightgbm as lgb
import joblib

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')

def setup_korean_font():
    """한글 폰트 설정"""
    import platform
    import matplotlib.font_manager as fm
    
    system = platform.system()
    
    if system == 'Windows':
        try:
            plt.rcParams['font.family'] = 'Malgun Gothic'
        except:
            try:
                font_path = 'C:/Windows/Fonts/malgun.ttf'
                font_name = fm.FontProperties(fname=font_path).get_name()
                plt.rc('font', family=font_name)
            except:
                print("⚠️ 한글 폰트 설정 실패")
    elif system == 'Darwin':
        plt.rcParams['font.family'] = 'AppleGothic'
    else:
        plt.rcParams['font.family'] = 'NanumGothic'
    
    plt.rcParams['axes.unicode_minus'] = False
    print("✅ 한글 폰트 설정 완료")

class LiverCancerTreatmentEffectPredictor:
    """간암 치료 효과 예측 모델 클래스 (XAI 포함)"""
    
    def __init__(self, data_path):
        setup_korean_font()
        self.data_path = data_path
        self.df = None
        self.processed_df = None
        self.models = {}
        self.results = {}
        self.feature_names = []
        self.shap_explainers = {}
        self.shap_values = {}
        self.lime_explainers = {}
        
        print(f"🚀 간암 치료 효과 예측 모델 초기화 (XAI 포함)")
        print(f"📁 데이터 경로: {data_path}")
        print(f"⏰ 시작 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print("="*60)
    
    def load_and_explore_data(self):
        """데이터 로드 및 탐색적 분석"""
        print("\n📊 1. 데이터 로드 및 탐색")
        
        try:
            self.df = pd.read_csv(self.data_path)
            print(f"✅ 데이터 로드 성공: {self.df.shape[0]}행 × {self.df.shape[1]}열")
        except Exception as e:
            print(f"❌ 데이터 로드 실패: {e}")
            return False
        
        # 기본 정보 출력
        print(f"📈 데이터 기본 정보:")
        print(f"   - 총 환자 수: {len(self.df)}")
        print(f"   - 총 컬럼 수: {len(self.df.columns)}")
        
        # 생존 상태 분포
        if 'vital_status' in self.df.columns:
            status_counts = self.df['vital_status'].value_counts()
            print(f"   - 생존 환자: {status_counts.get('Alive', 0)}명")
            print(f"   - 사망 환자: {status_counts.get('Dead', 0)}명")
            print(f"   - 사망률: {status_counts.get('Dead', 0)/len(self.df)*100:.1f}%")
        
        return True
    
    def preprocess_data(self):
        """데이터 전처리"""
        print("\n🔧 2. 데이터 전처리")
        
        # 치료 효과 예측 모델용 선택된 컬럼들[2]
        selected_columns = [
            # 생존 결과 변수
            'vital_status', 'days_to_death', 'days_to_last_follow_up',
            # 간기능 평가 (핵심 예측 인자)[3]
            'child_pugh_classification', 'ishak_fibrosis_score',
            # 병기 및 종양 특성[2]
            'ajcc_pathologic_stage', 'ajcc_pathologic_t', 'ajcc_pathologic_n', 'ajcc_pathologic_m',
            'tumor_grade', 'morphology',
            # 환자 기본 특성[2]
            'age_at_diagnosis', 'gender', 'race', 'ethnicity',
            # 치료 관련 변수
            'treatments_pharmaceutical_treatment_intent_type', 
            'treatments_pharmaceutical_treatment_type',
            'prior_treatment',
            # 추가 임상 변수
            'primary_diagnosis', 'year_of_diagnosis'
        ]
        
        # 존재하는 컬럼만 선택
        available_columns = [col for col in selected_columns if col in self.df.columns]
        missing_columns = [col for col in selected_columns if col not in self.df.columns]
        
        print(f"✅ 사용 가능한 컬럼: {len(available_columns)}개")
        if missing_columns:
            print(f"⚠️  누락된 컬럼: {missing_columns}")
        
        self.processed_df = self.df[available_columns].copy()
        
        # 생존 시간 및 이벤트 변수 생성
        print("🔄 생존 변수 생성 중...")
        self.processed_df['event'] = (self.processed_df['vital_status'] == 'Dead').astype(int)
        
        # 생존 시간 계산
        self.processed_df['duration'] = self.processed_df['days_to_death'].fillna(
            self.processed_df['days_to_last_follow_up']
        )
        
        # 치료 효과 지표 생성 (생존 시간 기반)
        self.processed_df['treatment_effectiveness'] = np.where(
            self.processed_df['duration'] > self.processed_df['duration'].median(), 1, 0
        )
        
        # 유효하지 않은 생존 시간 제거
        valid_mask = (self.processed_df['duration'].notna()) & (self.processed_df['duration'] > 0)
        self.processed_df = self.processed_df[valid_mask].copy()
        
        print(f"✅ 유효한 생존 데이터: {len(self.processed_df)}명")
        print(f"   - 사망 이벤트: {self.processed_df['event'].sum()}건")
        print(f"   - 중간 생존 시간: {self.processed_df['duration'].median():.0f}일")
        print(f"   - 치료 효과 양호: {self.processed_df['treatment_effectiveness'].sum()}명")
        
        # 결측값 분석
        print("\n📋 결측값 분석:")
        missing_analysis = self.processed_df.isnull().sum()
        missing_percent = (missing_analysis / len(self.processed_df) * 100).round(1)
        
        for col in missing_analysis[missing_analysis > 0].index:
            print(f"   - {col}: {missing_analysis[col]}개 ({missing_percent[col]}%)")
        
        # 높은 결측률 컬럼 제거 (80% 이상)
        high_missing_cols = missing_percent[missing_percent > 80].index.tolist()
        if high_missing_cols:
            print(f"🗑️  높은 결측률 컬럼 제거: {high_missing_cols}")
            self.processed_df = self.processed_df.drop(columns=high_missing_cols)
        
        return True
    
    def prepare_features(self):
        """특성 준비 및 인코딩 (NA와 Unknown 값 처리 포함)"""
        print("\n🎯 3. 특성 준비 및 인코딩")
        
        # 특성과 타겟 분리
        feature_cols = [col for col in self.processed_df.columns 
                       if col not in ['vital_status', 'days_to_death', 'days_to_last_follow_up', 
                                     'event', 'duration', 'treatment_effectiveness']]
        
        X = self.processed_df[feature_cols].copy()
        y_duration = self.processed_df['duration'].values
        y_event = self.processed_df['event'].values.astype(bool)
        y_effectiveness = self.processed_df['treatment_effectiveness'].values
        
        print(f"📊 초기 특성 개수: {len(feature_cols)}")
        print(f"📊 샘플 개수: {len(X)}")
        
        # 범주형 변수 인코딩
        categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
        numerical_cols = X.select_dtypes(include=[np.number]).columns.tolist()
        
        print(f"🔤 범주형 변수: {len(categorical_cols)}개")
        print(f"🔢 수치형 변수: {len(numerical_cols)}개")
        
        # 결측값 처리
        print("🔄 결측값 처리 중...")
        
        # 수치형 변수: 중앙값으로 대체
        if numerical_cols:
            num_imputer = SimpleImputer(strategy='median')
            X[numerical_cols] = num_imputer.fit_transform(X[numerical_cols])
        
        # 임상적으로 의미있는 Unknown 값을 가질 수 있는 컬럼들[3]
        meaningful_unknown_cols = [
            'child_pugh_classification', 'ajcc_pathologic_stage', 'ajcc_pathologic_t',
            'ajcc_pathologic_n', 'ajcc_pathologic_m', 'tumor_grade',
            'treatments_pharmaceutical_treatment_intent_type',
            'treatments_pharmaceutical_treatment_type', 'prior_treatment'
        ]
        
        label_encoders = {}
        
        for col in categorical_cols:
            if col in X.columns:
                print(f"\n   🔍 {col} 처리:")
                
                # 현재 값 분포 확인
                value_counts = X[col].value_counts(dropna=False)
                print(f"      - 전처리 전 분포: {dict(list(value_counts.items())[:3])}")
                
                # 'NA' 문자열을 결측치로 변환
                if 'NA' in X[col].values:
                    X[col] = X[col].replace('NA', np.nan)
                    print(f"      - 'NA' 문자열을 결측치로 변환")
                
                # Unknown 값 처리 결정
                has_unknown = X[col].str.contains('Unknown', na=False).any() if X[col].dtype == object else False
                
                if has_unknown:
                    if col in meaningful_unknown_cols:
                        print(f"      - 'Unknown' 값 유지 (임상적 의미 있음)")
                        if X[col].isnull().any():
                            mode_value = X[col].mode()
                            if not mode_value.empty:
                                fill_value = mode_value[0]
                                X[col] = X[col].fillna(fill_value)
                                print(f"      - 결측치를 '{fill_value}'로 대체")
                    else:
                        print(f"      - 'Unknown' 값을 결측치로 변환 후 대체")
                        X[col] = X[col].replace('Unknown', np.nan)
                        if X[col].isnull().any():
                            mode_value = X[col].mode()
                            if not mode_value.empty:
                                fill_value = mode_value[0]
                                X[col] = X[col].fillna(fill_value)
                                print(f"      - 결측치를 '{fill_value}'로 대체")
                else:
                    if X[col].isnull().any():
                        mode_value = X[col].mode()
                        if not mode_value.empty:
                            fill_value = mode_value[0]
                            X[col] = X[col].fillna(fill_value)
                            print(f"      - 결측치를 '{fill_value}'로 대체")
        
        # 모든 범주형 변수 인코딩
        print("\n🔄 범주형 변수 인코딩:")
        all_categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
        for col in all_categorical_cols:
            le = LabelEncoder()
            X[col] = le.fit_transform(X[col].astype(str))
            label_encoders[col] = le
            
            if col in meaningful_unknown_cols:
                mapping = dict(zip(le.classes_, le.transform(le.classes_)))
                print(f"   - {col} 인코딩 매핑: {mapping}")
        
        # 특성 스케일링
        scaler = StandardScaler()
        X_scaled = pd.DataFrame(
            scaler.fit_transform(X),
            columns=X.columns,
            index=X.index
        )
        
        self.feature_names = X_scaled.columns.tolist()
        
        # scikit-survival 형식으로 변환
        y_structured = np.array([(event, duration) for event, duration in zip(y_event, y_duration)],
                               dtype=[('event', '?'), ('time', '<f8')])
        
        print("✅ 특성 준비 완료")
        
        return X_scaled, y_structured, y_duration, y_event, y_effectiveness, scaler, label_encoders
    
    def split_data(self, X, y_structured, y_duration, y_event, y_effectiveness):
        """데이터 분할"""
        print("\n✂️  4. 데이터 분할 (훈련:검증:테스트 = 60:20:20)")
        
        # 먼저 훈련+검증 vs 테스트로 분할
        X_temp, X_test, y_temp_struct, y_test_struct, y_temp_dur, y_test_dur, y_temp_event, y_test_event, y_temp_eff, y_test_eff = \
            train_test_split(X, y_structured, y_duration, y_event, y_effectiveness,
                           test_size=0.2, random_state=42, stratify=y_effectiveness)
        
        # 훈련 vs 검증으로 분할
        X_train, X_val, y_train_struct, y_val_struct, y_train_dur, y_val_dur, y_train_event, y_val_event, y_train_eff, y_val_eff = \
            train_test_split(X_temp, y_temp_struct, y_temp_dur, y_temp_event, y_temp_eff,
                           test_size=0.25, random_state=42, stratify=y_temp_eff)
        
        print(f"📊 훈련 세트: {len(X_train)}명 (사망: {y_train_event.sum()}명, 치료효과양호: {y_train_eff.sum()}명)")
        print(f"📊 검증 세트: {len(X_val)}명 (사망: {y_val_event.sum()}명, 치료효과양호: {y_val_eff.sum()}명)")
        print(f"📊 테스트 세트: {len(X_test)}명 (사망: {y_test_event.sum()}명, 치료효과양호: {y_test_eff.sum()}명)")
        
        return (X_train, X_val, X_test, 
                y_train_struct, y_val_struct, y_test_struct,
                y_train_dur, y_val_dur, y_test_dur,
                y_train_event, y_val_event, y_test_event,
                y_train_eff, y_val_eff, y_test_eff)
    
    def train_models(self, X_train, X_val, X_test, 
                    y_train_struct, y_val_struct, y_test_struct,
                    y_train_dur, y_val_dur, y_test_dur,
                    y_train_event, y_val_event, y_test_event,
                    y_train_eff, y_val_eff, y_test_eff):
        """모델 훈련 (치료 효과 예측 특화)"""
        print("\n🤖 5. 모델 훈련")
        
        # 1. Random Survival Forest (생존 예측)[7]
        print("🔄 Random Survival Forest 훈련 중...")
        try:
            rsf_model = RandomSurvivalForest(
                n_estimators=100,
                max_depth=10,
                min_samples_split=10,
                min_samples_leaf=5,
                random_state=42,
                n_jobs=-1
            )
            rsf_model.fit(X_train, y_train_struct)
            self.models['RSF'] = rsf_model
            print("✅ Random Survival Forest 훈련 완료")
        except Exception as e:
            print(f"❌ RSF 모델 훈련 실패: {e}")
        
        # 2. Cox 비례위험 모델 (생존 예측)[7]
        print("🔄 Cox 비례위험 모델 훈련 중...")
        try:
            cox_model = CoxPHSurvivalAnalysis(alpha=0.5)
            cox_model.fit(X_train, y_train_struct)
            self.models['Cox_Survival'] = cox_model
            print("✅ Cox 생존 모델 훈련 완료")
        except Exception as e:
            print(f"❌ Cox 모델 훈련 실패: {e}")
        
        # 3. Gradient Boosting Survival Analysis
        print("🔄 Gradient Boosting Survival Analysis 훈련 중...")
        try:
            gbsa_model = GradientBoostingSurvivalAnalysis(
                n_estimators=100,
                learning_rate=0.1,
                max_depth=3,
                random_state=42
            )
            gbsa_model.fit(X_train, y_train_struct)
            self.models['GBSA'] = gbsa_model
            print("✅ Gradient Boosting Survival Analysis 훈련 완료")
        except Exception as e:
            print(f"❌ GBSA 모델 훈련 실패: {e}")
        
        # 4. Random Forest (치료 효과 분류)
        print("🔄 Random Forest 치료 효과 분류 모델 훈련 중...")
        try:
            rf_classifier = RandomForestClassifier(
                n_estimators=100,
                max_depth=10,
                random_state=42,
                n_jobs=-1
            )
            rf_classifier.fit(X_train, y_train_eff)
            self.models['RF_Treatment'] = rf_classifier
            print("✅ Random Forest 치료 효과 분류 모델 훈련 완료")
        except Exception as e:
            print(f"❌ RF 치료 효과 모델 훈련 실패: {e}")
        
        # 5. XGBoost (치료 효과 분류)[9]
        print("🔄 XGBoost 치료 효과 분류 모델 훈련 중...")
        try:
            xgb_model = xgb.XGBClassifier(
                n_estimators=100,
                max_depth=6,
                learning_rate=0.1,
                random_state=42,
                eval_metric='logloss'
            )
            xgb_model.fit(X_train, y_train_eff)
            self.models['XGB_Treatment'] = xgb_model
            print("✅ XGBoost 치료 효과 분류 모델 훈련 완료")
        except Exception as e:
            print(f"❌ XGBoost 치료 효과 모델 훈련 실패: {e}")
        
        # 6. LightGBM (치료 효과 분류)[9]
        print("🔄 LightGBM 치료 효과 분류 모델 훈련 중...")
        try:
            lgb_model = lgb.LGBMClassifier(
                n_estimators=100,
                max_depth=6,
                learning_rate=0.1,
                random_state=42,
                verbose=-1
            )
            lgb_model.fit(X_train, y_train_eff)
            self.models['LGB_Treatment'] = lgb_model
            print("✅ LightGBM 치료 효과 분류 모델 훈련 완료")
        except Exception as e:
            print(f"❌ LightGBM 치료 효과 모델 훈련 실패: {e}")
        
        # 7. Random Forest (생존 시간 회귀)
        print("🔄 Random Forest 생존 시간 회귀 모델 훈련 중...")
        try:
            rf_regressor = RandomForestRegressor(
                n_estimators=100,
                max_depth=10,
                random_state=42,
                n_jobs=-1
            )
            rf_regressor.fit(X_train, y_train_dur)
            self.models['RF_Duration'] = rf_regressor
            print("✅ Random Forest 생존 시간 회귀 모델 훈련 완료")
        except Exception as e:
            print(f"❌ RF 생존 시간 모델 훈련 실패: {e}")
        
        print(f"\n🎯 총 {len(self.models)}개 모델 훈련 완료")
        
        return True
    
    def evaluate_models(self, X_train, X_val, X_test,
                       y_train_struct, y_val_struct, y_test_struct,
                       y_train_dur, y_val_dur, y_test_dur,
                       y_train_event, y_val_event, y_test_event,
                       y_train_eff, y_val_eff, y_test_eff):
        """모델 평가"""
        print("\n📈 6. 모델 평가")
        
        datasets = {
            'Train': (X_train, y_train_struct, y_train_dur, y_train_event, y_train_eff),
            'Validation': (X_val, y_val_struct, y_val_dur, y_val_event, y_val_eff),
            'Test': (X_test, y_test_struct, y_test_dur, y_test_event, y_test_eff)
        }
        
        for model_name, model in self.models.items():
            print(f"\n🔍 {model_name} 모델 평가:")
            self.results[model_name] = {}
            
            for dataset_name, (X, y_struct, y_dur, y_event, y_eff) in datasets.items():
                try:
                    if model_name in ['RSF', 'Cox_Survival', 'GBSA']:
                        # 생존 모델 평가
                        risk_scores = model.predict(X)
                        c_index = concordance_index_censored(y_struct['event'], y_struct['time'], risk_scores)[0]
                        self.results[model_name][dataset_name] = {'c_index': c_index}
                        print(f"   {dataset_name}: C-index = {c_index:.3f}")
                    
                    elif model_name == 'RF_Duration':
                        # 회귀 모델 평가
                        y_pred = model.predict(X)
                        mse = mean_squared_error(y_dur, y_pred)
                        r2 = r2_score(y_dur, y_pred)
                        self.results[model_name][dataset_name] = {'mse': mse, 'r2': r2}
                        print(f"   {dataset_name}: MSE = {mse:.0f}, R² = {r2:.3f}")
                    
                    else:
                        # 분류 모델 평가 (치료 효과)
                        y_pred = model.predict(X)
                        accuracy = (y_pred == y_eff).mean()
                        
                        try:
                            y_proba = model.predict_proba(X)
                            auc_score = roc_auc_score(y_eff, y_proba[:, 1])
                        except:
                            auc_score = np.nan
                        
                        self.results[model_name][dataset_name] = {
                            'accuracy': accuracy,
                            'auc': auc_score
                        }
                        print(f"   {dataset_name}: Accuracy = {accuracy:.3f}, AUC = {auc_score:.3f}")
                    
                except Exception as e:
                    print(f"   ❌ {dataset_name} 평가 실패: {e}")
                    self.results[model_name][dataset_name] = {'error': str(e)}
        
        return True
    
    def explain_models(self, X_train, X_test):
        """XAI 모델 설명 생성[9]"""
        print("\n🔍 XAI 모델 설명 생성")
        
        # SHAP 설명기 초기화
        print("🔄 SHAP 설명 생성 중...")
        tree_models = ['RF_Treatment', 'XGB_Treatment', 'LGB_Treatment', 'RF_Duration']
        
        for model_name in tree_models:
            if model_name in self.models:
                try:
                    X_test_sample = X_test.iloc[:50]  # 처음 50개 샘플만 사용
                    
                    explainer = shap.TreeExplainer(self.models[model_name])
                    shap_values = explainer.shap_values(X_test_sample)
                    
                    self.shap_explainers[model_name] = explainer
                    self.shap_values[model_name] = shap_values
                    print(f"✅ {model_name} SHAP 설명 생성 완료")
                except Exception as e:
                    print(f"❌ {model_name} SHAP 실패: {e}")
        
        # LIME 설명기 초기화
        print("\n🔄 LIME 설명 생성 중...")
        classification_models = ['RF_Treatment', 'XGB_Treatment', 'LGB_Treatment']
        
        for model_name in classification_models:
            if model_name in self.models:
                try:
                    explainer = lime_tabular.LimeTabularExplainer(
                        training_data=X_train.values,
                        feature_names=self.feature_names,
                        class_names=['치료효과 불량', '치료효과 양호'],
                        mode='classification',
                        discretize_continuous=True
                    )
                    self.lime_explainers[model_name] = explainer
                    print(f"✅ {model_name} LIME 설명기 생성 완료")
                except Exception as e:
                    print(f"❌ {model_name} LIME 실패: {e}")
        
        return True
    
    def generate_xai_visualizations(self, X_test, sample_index=0):
        """XAI 시각화 생성 및 저장 (수정된 버전)"""
        print("\n📊 XAI 시각화 생성")
        
        # SHAP 시각화
        shap_figures = []
        for model_name in self.shap_explainers:
            try:
                print(f"🔄 {model_name} SHAP 시각화 디버깅:")
                
                shap_vals = self.shap_values[model_name]
                print(f"   - SHAP 값 타입: {type(shap_vals)}")
                print(f"   - SHAP 값 형태: {np.array(shap_vals).shape if isinstance(shap_vals, list) else shap_vals.shape}")
                
                # 이진 분류 SHAP 값 처리 (핵심 수정 부분)
                if isinstance(shap_vals, list) and len(shap_vals) == 2:
                    # 이진 분류: 양성 클래스(인덱스 1)만 사용
                    shap_vals_to_plot = shap_vals[1]
                    expected_val = self.shap_explainers[model_name].expected_value[1]
                    print(f"   - 이진 분류 감지: 양성 클래스 SHAP 값 사용")
                    print(f"   - 양성 클래스 SHAP 형태: {shap_vals_to_plot.shape}")
                    
                elif hasattr(shap_vals, 'shape') and shap_vals.ndim == 3:
                    # 3차원 배열: (샘플, 특성, 클래스)
                    shap_vals_to_plot = shap_vals[:, :, 1]  # 양성 클래스
                    expected_val = (self.shap_explainers[model_name].expected_value[1] 
                                if hasattr(self.shap_explainers[model_name].expected_value, '__len__') 
                                else self.shap_explainers[model_name].expected_value)
                    print(f"   - 3차원 배열 감지: 양성 클래스 선택")
                    
                else:
                    # 2차원 배열 또는 기타
                    shap_vals_to_plot = shap_vals
                    expected_val = (self.shap_explainers[model_name].expected_value 
                                if not hasattr(self.shap_explainers[model_name].expected_value, '__len__')
                                else self.shap_explainers[model_name].expected_value[0])
                    print(f"   - 2차원 배열 사용")
                
                print(f"   - 최종 SHAP 형태: {shap_vals_to_plot.shape}")
                print(f"   - 기댓값: {expected_val}")
                
                # Summary plot
                plt.figure(figsize=(10,6))
                shap.summary_plot(shap_vals_to_plot, X_test.iloc[:50], 
                                feature_names=self.feature_names,
                                plot_type="bar", show=False)
                plt.title(f"{model_name} 특성 중요도 (SHAP)")
                shap_summary_path = f"shap_summary_{model_name}.png"
                plt.savefig(shap_summary_path, bbox_inches='tight')
                plt.close()
                
                # Individual waterfall plot
                plt.figure(figsize=(12,6))
                
                # 개별 샘플 SHAP 값 추출 (스칼라 변환 오류 방지)
                if shap_vals_to_plot.ndim == 2:
                    individual_shap = shap_vals_to_plot[sample_index, :]  # 명시적 인덱싱
                else:
                    individual_shap = shap_vals_to_plot
                
                print(f"   - 개별 SHAP 형태: {individual_shap.shape}")
                
                # 안전한 기댓값 처리
                if hasattr(expected_val, '__len__') and len(expected_val) > 0:
                    base_value = float(expected_val[0]) if hasattr(expected_val[0], '__float__') else 0.0
                else:
                    base_value = float(expected_val) if hasattr(expected_val, '__float__') else 0.0
                
                print(f"   - 기댓값 (스칼라): {base_value}")
                
                # Waterfall plot 생성
                shap.waterfall_plot(
                    shap.Explanation(
                        values=individual_shap.astype(float),  # 명시적 float 변환
                        base_values=base_value,
                        data=X_test.iloc[sample_index].values.astype(float),  # 명시적 float 변환
                        feature_names=self.feature_names
                    ),
                    show=False
                )
                plt.title(f"{model_name} 개별 설명 (샘플 {sample_index})")
                shap_waterfall_path = f"shap_waterfall_{model_name}_{sample_index}.png"
                plt.savefig(shap_waterfall_path, bbox_inches='tight')
                plt.close()
                
                shap_figures.extend([shap_summary_path, shap_waterfall_path])
                print(f"✅ {model_name} SHAP 시각화 완료")
                
            except Exception as e:
                print(f"❌ {model_name} SHAP 시각화 실패: {e}")
                print(f"   상세 오류 정보:")
                import traceback
                traceback.print_exc()
        
        # LIME 시각화는 기존과 동일
        lime_figures = []
        for model_name in self.lime_explainers:
            try:
                exp = self.lime_explainers[model_name].explain_instance(
                    X_test.iloc[sample_index].values,
                    self.models[model_name].predict_proba,
                    num_features=5
                )
                lime_path = f"lime_explanation_{model_name}_{sample_index}.png"
                fig = exp.as_pyplot_figure()
                plt.title(f"{model_name} LIME 설명 (샘플 {sample_index})")
                plt.savefig(lime_path, bbox_inches='tight')
                plt.close()
                lime_figures.append(lime_path)
                print(f"✅ {model_name} LIME 시각화 완료")
            except Exception as e:
                print(f"❌ {model_name} LIME 시각화 실패: {e}")
        
        return shap_figures, lime_figures

    def plot_results(self, X_test, y_test_dur, y_test_event, y_test_eff):
        """결과 시각화 (치료 효과 예측 특화)"""
        print("\n📊 7. 결과 시각화")
        
        fig = plt.figure(figsize=(25, 35))
        gs = fig.add_gridspec(5, 3)
        axes = [
            fig.add_subplot(gs[0, 0]),  # 치료효과별 생존 곡선
            fig.add_subplot(gs[0, 1]),  # 치료효과 분포
            fig.add_subplot(gs[0, 2]),  # 생존 모델 성능
            fig.add_subplot(gs[1, 0]),  # 치료효과 분류 성능
            fig.add_subplot(gs[1, 1]),  # 회귀 모델 성능
            fig.add_subplot(gs[1, 2]),  # 특성 중요도 (XGBoost)
            fig.add_subplot(gs[2, 0]),  # 특성 중요도 (LightGBM)
            fig.add_subplot(gs[2, 1]),  # 치료효과별 생존율
            fig.add_subplot(gs[2, 2]),  # 모델 성능 종합
            fig.add_subplot(gs[3, :]),  # SHAP 시각화
            fig.add_subplot(gs[4, :])   # LIME 시각화
        ]
        
        # 1. 치료효과별 생존 곡선
        print("🔍 치료효과별 생존 곡선 생성 중...")
        for eff_level in [0, 1]:
            mask = (y_test_eff == eff_level)
            if mask.sum() > 5:
                kmf = KaplanMeierFitter()
                label = '치료효과 양호' if eff_level == 1 else '치료효과 불량'
                kmf.fit(y_test_dur[mask], y_test_event[mask], label=f'{label} (n={mask.sum()})')
                kmf.plot_survival_function(ax=axes[0])
        
        axes[0].set_title('치료효과별 생존 곡선 (Kaplan-Meier)')
        axes[0].set_ylabel('생존 확률')
        axes[0].set_xlabel('시간 (일)')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # 2. 치료효과 분포
        eff_counts = pd.Series(y_test_eff).value_counts().sort_index()
        colors = ['lightcoral', 'lightgreen']
        labels = ['치료효과 불량', '치료효과 양호']
        bars = axes[1].bar([labels[i] for i in eff_counts.index], 
                          eff_counts.values, color=colors)
        axes[1].set_title('테스트 세트 치료효과 분포')
        axes[1].set_ylabel('환자 수')
        
        for bar, value in zip(bars, eff_counts.values):
            axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                        f'{value}명', ha='center', va='bottom', fontweight='bold')
        
        # 3. 생존 모델 성능 비교
        survival_models = ['RSF', 'Cox_Survival', 'GBSA']
        survival_c_indices = []
        survival_names = []
        
        for model_name in survival_models:
            if model_name in self.results and 'Test' in self.results[model_name]:
                c_index = self.results[model_name]['Test'].get('c_index', np.nan)
                if not np.isnan(c_index):
                    survival_c_indices.append(c_index)
                    survival_names.append(model_name)
        
        if survival_c_indices:
            bars = axes[2].bar(survival_names, survival_c_indices, 
                             color=['skyblue', 'lightcoral', 'lightgreen'][:len(survival_names)])
            axes[2].set_title('생존 모델 성능 (C-index)')
            axes[2].set_ylabel('C-index')
            axes[2].set_ylim(0.5, 1.0)
            axes[2].grid(True, alpha=0.3, axis='y')
            
            for bar, value in zip(bars, survival_c_indices):
                axes[2].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                            f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
        
        # 4. 치료효과 분류 모델 성능
        classification_models = ['RF_Treatment', 'XGB_Treatment', 'LGB_Treatment']
        classification_accuracies = []
        classification_names = []
        
        for model_name in classification_models:
            if model_name in self.results and 'Test' in self.results[model_name]:
                accuracy = self.results[model_name]['Test'].get('accuracy', np.nan)
                if not np.isnan(accuracy):
                    classification_accuracies.append(accuracy)
                    classification_names.append(model_name)
        
        if classification_accuracies:
            bars = axes[3].bar(classification_names, classification_accuracies, 
                             color=['lightgreen', 'orange', 'purple'][:len(classification_names)])
            axes[3].set_title('치료효과 분류 모델 성능 (Accuracy)')
            axes[3].set_ylabel('Accuracy')
            axes[3].set_ylim(0, 1.0)
            axes[3].grid(True, alpha=0.3, axis='y')
            
            for bar, value in zip(bars, classification_accuracies):
                axes[3].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                            f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
        
        # 5. 회귀 모델 성능
        if 'RF_Duration' in self.results and 'Test' in self.results['RF_Duration']:
            r2 = self.results['RF_Duration']['Test'].get('r2', np.nan)
            mse = self.results['RF_Duration']['Test'].get('mse', np.nan)
            
            if not np.isnan(r2):
                bars = axes[4].bar(['RF_Duration'], [r2], color='gold')
                axes[4].set_title('생존시간 예측 모델 성능 (R²)')
                axes[4].set_ylabel('R²')
                axes[4].set_ylim(0, 1.0)
                axes[4].grid(True, alpha=0.3, axis='y')
                
                axes[4].text(bars[0].get_x() + bars[0].get_width()/2, bars[0].get_height() + 0.01,
                            f'{r2:.3f}', ha='center', va='bottom', fontweight='bold')
        
        # 6-7. 특성 중요도 시각화
        for idx, model_name in enumerate(['XGB_Treatment', 'LGB_Treatment']):
            ax_idx = 5 + idx
            if model_name in self.models:
                try:
                    importance = self.models[model_name].feature_importances_
                    feature_importance_df = pd.DataFrame({
                        'feature': self.feature_names,
                        'importance': importance
                    }).sort_values('importance', ascending=True).tail(10)
                    
                    color = 'orange' if 'XGB' in model_name else 'purple'
                    bars = axes[ax_idx].barh(range(len(feature_importance_df)), 
                                           feature_importance_df['importance'],
                                           color=color)
                    
                    axes[ax_idx].set_yticks(range(len(feature_importance_df)))
                    axes[ax_idx].set_yticklabels(feature_importance_df['feature'], fontsize=10)
                    axes[ax_idx].set_title(f'특성 중요도 ({model_name})', fontsize=12, fontweight='bold')
                    axes[ax_idx].set_xlabel('중요도')
                    axes[ax_idx].grid(True, alpha=0.3, axis='x')
                    
                except Exception as e:
                    print(f"❌ {model_name} 특성 중요도 시각화 실패: {e}")
        
        # 8. 치료효과별 생존율 요약
        survival_summary = []
        time_points = [365, 1095, 1825]  # 1년, 3년, 5년
        
        for eff_level in [0, 1]:
            mask = (y_test_eff == eff_level)
            if mask.sum() > 5:
                kmf = KaplanMeierFitter()
                kmf.fit(y_test_dur[mask], y_test_event[mask])
                
                survival_rates = []
                for time_point in time_points:
                    try:
                        survival_rate = kmf.survival_function_at_times(time_point).values[0]
                        survival_rates.append(survival_rate * 100)
                    except:
                        survival_rates.append(np.nan)
                
                survival_summary.append(survival_rates)
            else:
                survival_summary.append([np.nan, np.nan, np.nan])
        
        # 생존율 히트맵
        survival_df = pd.DataFrame(survival_summary, 
                                  index=['치료효과 불량', '치료효과 양호'],
                                  columns=['1년', '3년', '5년'])
        
        sns.heatmap(survival_df, annot=True, fmt='.1f', cmap='RdYlGn', 
                   ax=axes[7], cbar_kws={'label': '생존율 (%)'})
        axes[7].set_title('치료효과별 생존율 요약')
        axes[7].set_ylabel('치료효과 그룹')
        
        # 9. 모델 성능 종합
        all_models_performance = {}
        
        # 생존 모델 C-index
        for model_name in ['RSF', 'Cox_Survival', 'GBSA']:
            if model_name in self.results and 'Test' in self.results[model_name]:
                c_index = self.results[model_name]['Test'].get('c_index', np.nan)
                if not np.isnan(c_index):
                    all_models_performance[f'{model_name}_C-index'] = c_index
        
        # 분류 모델 Accuracy
        for model_name in ['RF_Treatment', 'XGB_Treatment', 'LGB_Treatment']:
            if model_name in self.results and 'Test' in self.results[model_name]:
                accuracy = self.results[model_name]['Test'].get('accuracy', np.nan)
                if not np.isnan(accuracy):
                    all_models_performance[f'{model_name}_Accuracy'] = accuracy
        
        if all_models_performance:
            model_names = list(all_models_performance.keys())
            model_scores = list(all_models_performance.values())
            
            bars = axes[8].bar(range(len(model_names)), model_scores, 
                             color=['skyblue', 'lightcoral', 'lightgreen', 
                                   'gold', 'orange', 'purple'][:len(model_names)])
            axes[8].set_title('전체 모델 성능 종합')
            axes[8].set_ylabel('성능 점수')
            axes[8].set_xticks(range(len(model_names)))
            axes[8].set_xticklabels(model_names, rotation=45, ha='right')
            axes[8].grid(True, alpha=0.3, axis='y')
            
            for bar, value in zip(bars, model_scores):
                axes[8].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                            f'{value:.3f}', ha='center', va='bottom', fontweight='bold', fontsize=8)
        
        # 10-11. XAI 시각화 로드
        try:
            # SHAP 시각화
            model_name = 'XGB_Treatment' if 'XGB_Treatment' in self.shap_explainers else 'RF_Treatment'
            if model_name:
                img = plt.imread(f"shap_summary_{model_name}.png")
                axes[9].imshow(img)
                axes[9].axis('off')
                axes[9].set_title('SHAP 전역 설명 (치료효과 예측)', fontsize=12)
        except Exception as e:
            print(f"❌ SHAP 시각화 로드 실패: {e}")
            axes[9].text(0.5, 0.5, 'SHAP 시각화 불러오기 실패', 
                        ha='center', va='center', transform=axes[9].transAxes,
                        fontsize=12, fontweight='bold')
        
        try:
            # LIME 시각화
            model_name = 'XGB_Treatment' if 'XGB_Treatment' in self.lime_explainers else 'RF_Treatment'
            if model_name:
                img = plt.imread(f"lime_explanation_{model_name}_0.png")
                axes[10].imshow(img)
                axes[10].axis('off')
                axes[10].set_title('LIME 개별 설명 (첫 번째 환자)', fontsize=12)
        except Exception as e:
            print(f"❌ LIME 시각화 로드 실패: {e}")
            axes[10].text(0.5, 0.5, 'LIME 시각화 불러오기 실패', 
                         ha='center', va='center', transform=axes[10].transAxes,
                         fontsize=12, fontweight='bold')
        
        plt.tight_layout()
        save_path = "liver_cancer_treatment_effect_prediction_with_xai.png"
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"📁 시각화 결과 저장: {save_path}")
        
        return True
    
    def generate_report(self):
        """결과 보고서 생성"""
        print("\n📋 8. 결과 보고서 생성")
        
        report = []
        report.append("="*60)
        report.append("간암 환자 치료 효과 예측 모델 분석 결과")
        report.append("="*60)
        report.append(f"분석 일시: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        report.append(f"데이터 경로: {self.data_path}")
        report.append("")
        
        # 데이터 요약
        report.append("📊 데이터 요약")
        report.append("-" * 30)
        report.append(f"총 환자 수: {len(self.processed_df)}")
        report.append(f"사망 환자 수: {self.processed_df['event'].sum()}")
        report.append(f"사망률: {self.processed_df['event'].mean()*100:.1f}%")
        report.append(f"중간 생존 시간: {self.processed_df['duration'].median():.0f}일")
        report.append(f"치료효과 양호 환자: {self.processed_df['treatment_effectiveness'].sum()}명")
        report.append(f"치료효과 양호율: {self.processed_df['treatment_effectiveness'].mean()*100:.1f}%")
        report.append(f"사용된 특성 수: {len(self.feature_names)}")
        report.append("")
        
        # 모델 성능
        report.append("🤖 모델 성능")
        report.append("-" * 30)
        
        # 생존 모델 성능
        report.append("\n생존 예측 모델 (C-index):")
        survival_models = ['RSF', 'Cox_Survival', 'GBSA']
        for model_name in survival_models:
            if model_name in self.results:
                for dataset in ['Train', 'Validation', 'Test']:
                    if dataset in self.results[model_name]:
                        metrics = self.results[model_name][dataset]
                        if 'c_index' in metrics:
                            report.append(f"  {model_name} {dataset}: C-index = {metrics['c_index']:.3f}")
        
        # 치료효과 분류 모델 성능
        report.append("\n치료효과 분류 모델 (Accuracy):")
        classification_models = ['RF_Treatment', 'XGB_Treatment', 'LGB_Treatment']
        for model_name in classification_models:
            if model_name in self.results:
                for dataset in ['Train', 'Validation', 'Test']:
                    if dataset in self.results[model_name]:
                        metrics = self.results[model_name][dataset]
                        if 'accuracy' in metrics:
                            report.append(f"  {model_name} {dataset}: Accuracy = {metrics['accuracy']:.3f}")
        
        # 회귀 모델 성능
        report.append("\n생존시간 예측 모델 (R²):")
        if 'RF_Duration' in self.results:
            for dataset in ['Train', 'Validation', 'Test']:
                if dataset in self.results['RF_Duration']:
                    metrics = self.results['RF_Duration'][dataset]
                    if 'r2' in metrics:
                        report.append(f"  RF_Duration {dataset}: R² = {metrics['r2']:.3f}")
        
        # 최고 성능 모델
        test_performances = {}
        for model_name in self.results:
            if 'Test' in self.results[model_name]:
                metrics = self.results[model_name]['Test']
                if 'c_index' in metrics:
                    test_performances[f"{model_name}_C-index"] = metrics['c_index']
                elif 'accuracy' in metrics:
                    test_performances[f"{model_name}_Accuracy"] = metrics['accuracy']
                elif 'r2' in metrics:
                    test_performances[f"{model_name}_R²"] = metrics['r2']
        
        if test_performances:
            best_model = max(test_performances, key=test_performances.get)
            report.append(f"\n🏆 최고 성능 모델: {best_model} (점수: {test_performances[best_model]:.3f})")
        
        report.append("")
        report.append("ℹ️  사용된 모델:")
        report.append("   생존 예측: Random Survival Forest, Cox, GBSA")
        report.append("   치료효과 분류: Random Forest, XGBoost, LightGBM")
        report.append("   생존시간 예측: Random Forest Regressor")
        report.append("   XAI: SHAP, LIME 적용")
        report.append("")
        report.append("="*60)
        
        # 보고서 저장
        report_text = "\n".join(report)
        with open("liver_cancer_treatment_effect_prediction_report.txt", "w", encoding="utf-8") as f:
            f.write(report_text)
        
        print("📁 보고서 저장: liver_cancer_treatment_effect_prediction_report.txt")
        print("\n" + report_text)
        
        return report_text
    
    def save_models(self):
        """모델 저장"""
        print("\n💾 9. 모델 저장")
        
        for model_name, model in self.models.items():
            try:
                filename = f"liver_cancer_treatment_{model_name.lower()}_model.pkl"
                joblib.dump(model, filename)
                print(f"✅ {model_name} 모델 저장: {filename}")
            except Exception as e:
                print(f"❌ {model_name} 모델 저장 실패: {e}")
        
        return True
    
    def run_complete_analysis(self):
        """전체 분석 실행 (치료 효과 예측 XAI 포함)"""
        print("🎯 간암 치료 효과 예측 모델 전체 분석 시작 (XAI 포함)")
        
        try:
            # 1. 데이터 로드
            if not self.load_and_explore_data():
                return False
            
            # 2. 데이터 전처리
            if not self.preprocess_data():
                return False
            
            # 3. 특성 준비
            X, y_structured, y_duration, y_event, y_effectiveness, scaler, label_encoders = self.prepare_features()
            
            # 4. 데이터 분할
            (X_train, X_val, X_test, 
             y_train_struct, y_val_struct, y_test_struct,
             y_train_dur, y_val_dur, y_test_dur,
             y_train_event, y_val_event, y_test_event,
             y_train_eff, y_val_eff, y_test_eff) = self.split_data(X, y_structured, y_duration, y_event, y_effectiveness)
            
            # 5. 모델 훈련
            if not self.train_models(X_train, X_val, X_test,
                                   y_train_struct, y_val_struct, y_test_struct,
                                   y_train_dur, y_val_dur, y_test_dur,
                                   y_train_event, y_val_event, y_test_event,
                                   y_train_eff, y_val_eff, y_test_eff):
                return False
            
            # 6. 모델 평가
            if not self.evaluate_models(X_train, X_val, X_test,
                                       y_train_struct, y_val_struct, y_test_struct,
                                       y_train_dur, y_val_dur, y_test_dur,
                                       y_train_event, y_val_event, y_test_event,
                                       y_train_eff, y_val_eff, y_test_eff):
                return False
            
            # 7. XAI 설명 생성
            self.explain_models(X_train, X_test)
            
            # 8. XAI 시각화 생성
            self.generate_xai_visualizations(X_test)
            
            # 9. 시각화 (XAI 포함)
            if not self.plot_results(X_test, y_test_dur, y_test_event, y_test_eff):
                return False
            
            # 10. 보고서 생성
            self.generate_report()
            
            # 11. 모델 저장
            self.save_models()
            
            print("\n🎉 전체 분석 완료!")
            print(f"⏰ 완료 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            
            return True
            
        except Exception as e:
            print(f"\n❌ 분석 중 오류 발생: {e}")
            import traceback
            traceback.print_exc()
            return False

# 실행
if __name__ == "__main__":
    # 데이터 경로
    data_path = r"C:\Users\02\Documents\GDCdata_liver\clinical_data_liver.csv"
    
    # 분석 실행
    treatment_model = LiverCancerTreatmentEffectPredictor(data_path)
    success = treatment_model.run_complete_analysis()
    
    if success:
        print("\n✨ 모든 분석이 성공적으로 완료되었습니다!")
        print("📁 생성된 파일들:")
        print("   - liver_cancer_treatment_effect_prediction_with_xai.png (시각화 결과)")
        print("   - liver_cancer_treatment_effect_prediction_report.txt (분석 보고서)")
        print("   - liver_cancer_treatment_*_model.pkl (훈련된 모델들)")
        print("   - shap_*.png (SHAP 설명)")
        print("   - lime_*.png (LIME 설명)")
        print("\n🔬 포함된 모델들:")
        print("   생존 예측: Random Survival Forest, Cox, GBSA")
        print("   치료효과 분류: Random Forest, XGBoost, LightGBM")
        print("   생존시간 예측: Random Forest Regressor")
        print("   XAI: SHAP, LIME 적용")
        print("\n🎯 임상 활용:")
        print("   - 개별화된 치료 계획 수립")
        print("   - 치료 반응 예측 및 모니터링")
        print("   - 고위험군 집중 관리 전략")
    else:
        print("\n❌ 분석 중 문제가 발생했습니다. 로그를 확인해주세요.")


✅ 한글 폰트 설정 완료
🚀 간암 치료 효과 예측 모델 초기화 (XAI 포함)
📁 데이터 경로: C:\Users\02\Documents\GDCdata_liver\clinical_data_liver.csv
⏰ 시작 시간: 2025-06-13 15:55:23
🎯 간암 치료 효과 예측 모델 전체 분석 시작 (XAI 포함)

📊 1. 데이터 로드 및 탐색
✅ 데이터 로드 성공: 377행 × 87열
📈 데이터 기본 정보:
   - 총 환자 수: 377
   - 총 컬럼 수: 87
   - 생존 환자: 245명
   - 사망 환자: 132명
   - 사망률: 35.0%

🔧 2. 데이터 전처리
✅ 사용 가능한 컬럼: 20개
🔄 생존 변수 생성 중...
✅ 유효한 생존 데이터: 372명
   - 사망 이벤트: 132건
   - 중간 생존 시간: 602일
   - 치료 효과 양호: 188명

📋 결측값 분석:
   - days_to_death: 240개 (64.5%)
   - child_pugh_classification: 47개 (12.6%)
   - ishak_fibrosis_score: 96개 (25.8%)
   - ajcc_pathologic_stage: 24개 (6.5%)
   - ajcc_pathologic_t: 2개 (0.5%)
   - ajcc_pathologic_n: 1개 (0.3%)
   - tumor_grade: 5개 (1.3%)
   - age_at_diagnosis: 3개 (0.8%)
   - treatments_pharmaceutical_treatment_intent_type: 44개 (11.8%)
   - treatments_pharmaceutical_treatment_type: 42개 (11.3%)
   - year_of_diagnosis: 2개 (0.5%)

🎯 3. 특성 준비 및 인코딩
📊 초기 특성 개수: 17
📊 샘플 개수: 372
🔤 범주형 변수: 15개
🔢 수치형 변수: 2개
🔄 결측값 처리 중...

   🔍 child_pugh_cl

## LIME 개별 환자 설명 (XGBoost 모델)

- 치료 효과에 영향을 미친 요인들:

1) 치료 효과 양호 방향 (녹색):
- **year_of_diagnosis ≤ -0.45**: 진단 연도가 치료 효과 향상에 가장 큰 기여 (0.15)
- **tumor_grade ≤ -0.33**: 종양 등급이 낮을수록 치료 효과 양호 (0.13)
- **gender ≤ -1.44**: 성별 요인이 치료 효과에 긍정적 기여 (0.05)

2) 치료 효과 불량 방향 (빨간색):
- **-0.67 < ajcc_pathologic_n ≤ 1.50**: 림프절 전이 상태가 치료 효과에 부정적 영향 (-0.07)
- **ishak_fibrosis_score ≤ -0.68**: 간섬유화 점수가 치료 효과에 부정적 영향 (-0.05)

## SHAP 전역 특성 중요도 (LightGBM 모델)

1) 가장 중요한 치료 효과 예측 인자들:
1. **year_of_diagnosis** (1.5): 진단 연도가 압도적으로 가장 중요한 인자
2. **age_at_diagnosis** (0.5): 진단 시 연령
3. **ishak_fibrosis_score** (0.5): 간섬유화 정도
4. **tumor_grade** (0.4): 종양 등급
5. **race** (0.3): 인종적 요인

## 모델 성능 분석

1) 성능:
- **최고 성능 모델**: XGBoost (테스트 정확도 66.7%)
- **생존 예측**: Cox 모델 (C-index 0.648)이 가장 안정적
- **치료 효과 분류**: 모든 모델이 60-67% 수준의 적정 성능

## 임상적 해석

1) 진단 연도의 압도적 중요성:
- 의료 기술 발전, 치료법 개선, 조기 진단 기법 향상 등이 복합적으로 작용
- 최근 진단받은 환자일수록 치료 효과가 양호할 가능성 높음
- 이는 간암 치료의 지속적인 발전을 반영

2) 연령과 종양 특성의 역할:
- 젊은 연령에서 치료 반응이 더 좋음
- 낮은 종양 등급(well-differentiated)에서 치료 효과 우수
- 간섬유화 정도가 치료 결과에 중요한 영향

3) 개별화된 치료 전략:
- 진단 연도, 연령, 종양 등급을 종합적으로 고려한 치료 계획 수립 필요
- 고령 환자나 고등급 종양 환자에서는 더 적극적인 치료 접근 고려
- 간섬유화가 심한 환자에서는 간기능 보존을 우선한 치료 전략

## XAI의 임상적 가치

1) 개별 환자 맞춤 설명:
- LIME 결과를 통해 각 환자별로 어떤 인자가 치료 효과에 가장 큰 영향을 미쳤는지 명확히 파악
- 의료진이 해당 인자들을 집중적으로 관리할 수 있는 근거 제공

2) 치료 의사결정 지원:
- 진단 연도와 환자 특성을 고려한 개별화된 치료 강도 결정
- 치료 반응 예측을 통한 조기 치료 전략 수정
